In [1]:
import arcpy
import os
from arcpy import env
from arcpy.sa import *

### Set workspace for saving outputs

In [2]:
# set input and output folder, create output file geodatabase
mainInputFolder = "C:\\Users\\Zachary\\ASSET\\resourceAssessment\\analysis\\data" # enter path to input folder
mainOutputFolder = "C:\\Users\\Zachary\\ASSET\\resourceAssessment\\analysis\\data\\" # enter path to output folder


gdbFileName = "resourceAssessmentResults.gdb" # enter name of output geodatabase
scratch_gdb = os.path.join(mainOutputFolder, "scratch.gdb")

In [3]:
# set scratchworkspace to output workspace
if not arcpy.Exists(scratch_gdb):
    arcpy.CreateFileGDB_management(mainOutputFolder, "scratch.gdb")
    print("Scratch geodatabase created successfully.")

env.scratchWorkspace = scratch_gdb # creates scratch geodatabase in output folder

In [4]:
# set environment workspace to output workspace
env.workspace = os.path.join(mainOutputFolder, gdbFileName) 

In [95]:
# set input paths:
grid = os.path.join(mainInputFolder, "grids.gdb\\cesm2_base_grid") # enter path to grid shapefile
cpa = os.path.join(mainInputFolder, "resourceAssessment.gdb\\Candidate_ProjectArea_Solar_SL1_FullWest_Wu" ) # enter path to inclusion area/cpa shapefile


In [96]:
# set intermediate output paths to scratch geodatabase
output_dissolved = os.path.join(env.scratchWorkspace, "SL1_SolarCandidateProjectAreas_cesm2_dissolved")  # enter path to dissolved layer
output_intersected = os.path.join(env.scratchWorkspace, "SL1_SolarCandidateProjectAreas_cesm2_intersected")  # enter path to intersected layer
output_centroid = os.path.join(env.scratchWorkspace, "SL1_SolarCandidateProjectAreas_cesm2_centroid") # enter path to centroid layer
output_join = os.path.join(env.workspace, "SL1_SolarCandidateProjectAreas_cesm2_areaAssessment")  # Replace with the desired name for the joined layer

### Dissolve, Intersect
Dissolve the Candidate Project Area/Inclusion Area. Intersect the dissolved layer with the grid layer (cesm2 or era5) to create polygons for each grid cell.

In [97]:
# Repair geometry of the input layer
try:
    # Enable overwrite output
    arcpy.env.overwriteOutput = True
    
    # Check if input feature class exists
    if arcpy.Exists(cpa):
        # Repair the geometry of the feature class
        arcpy.RepairGeometry_management(cpa)
        
        print("Geometry repaired successfully.")
    else:
        print("Input feature class does not exist.")
        
except arcpy.ExecuteError:
    print(arcpy.GetMessages())

Geometry repaired successfully.


In [ ]:
# Dissolve the first polygon layer
arcpy.Dissolve_management(cpa, output_dissolved) # get a coffee while this runs

In [ ]:
# Intersect the dissolved layer with the second polygon layer
arcpy.Intersect_analysis([output_dissolved, grid], output_intersected) # get a cookie while this runs

### Add Area field and calculate area
Add an attribute called 'SIArea' to the intersected layer attribute table. Calculate the area for each polygon.

In [ ]:
# Add an "Area" field to the output_intersected layer
arcpy.AddField_management(output_intersected, "CPA_Area", "DOUBLE")

# Calculate the area for all polygons and populate the "Area" field
expression = "!SHAPE.AREA!"
arcpy.CalculateField_management(output_intersected, "CPA_Area", expression, "PYTHON")

### Feature to Point 
Convert the intersected layer to a point layer for subsequent 'join attributes by location' with grid layer. 

In [ ]:
# Create a point layer within the original features
arcpy.FeatureToPoint_management(output_intersected, output_centroid, "INSIDE")

### Join attributes by location
Joins the centroid layer containing area calculations to the original grid dataset. 

In [ ]:
# Perform a spatial join to transfer attributes from input_layer2 to the output_points layer
arcpy.SpatialJoin_analysis(grid, output_centroid, output_join)

In [ ]:
# Add a 'pct' field to the output_join layer
arcpy.AddField_management(os.path.join(env.workspace, "SL1_Wind_era5_areaAssessment"), "IA_pct", "DOUBLE")

# Calculate the area for all polygons and populate the "pct" field
num = 'IA_Area'
denom = 'Shape_Area'
expression = "None if (!{0}! is None or !{1}! is None or !{1}! == 0) else round(!{0}! / !{1}!, 2)".format(
            num, denom)
arcpy.CalculateField_management(os.path.join(env.workspace, "SL1_Wind_era5_areaAssessment"), "IA_pct", expression, "PYTHON")

### View and modify field names and export to output file geodatabase

In [ ]:
# Print the field names of the output_join feature class
field_names = [field.name for field in arcpy.ListFields(output_join)]
print("Field names:", field_names)

In [ ]:
# # Delete a specific attribute field from the output_join feature class
# fields_to_delete = ['Join_Count', 'TARGET_FID', 'FID_1', 'FID_SL1_SolarCandidateProjectAreas_cesm2_dissolved', 'FID_cesm2_base_grid', 'FID_12']  # Replace with the name of the field to delete
# if arcpy.Exists(output_join):
#     arcpy.DeleteField_management(output_join, fields_to_delete)
#     print(f"The field '{fields_to_delete}' has been deleted.")

### Join 

In [26]:
output_synthesis = os.path.join(env.workspace, "SL1_Wind_cesm2_areaAssessment")

In [ ]:
# Add a 'dif' field to the synthesis layer
arcpy.AddField_management(output_synthesis, "pct_dif", "DOUBLE")

# Calculate the area for all polygons and populate the "dif" field
zrm = 'IA_Area'
wu = 'CPA_Area'
denom = 'Shape_Area'
expression = "None if (!{0}! is None or !{1}! is None or !{1}! == 0) else round((!{0}! - !{1}!) / !{2}!, 4)".format(
            zrm, wu, denom)
arcpy.CalculateField_management(output_synthesis, "pct_dif", expression, "PYTHON")